In [25]:
'''
CS233 Stanford, HW4

Note: Code tested with Python 2.7.

Written by Panos Achlioptas, 2018.
'''

import numpy as np
import os.path as osp
from collections import defaultdict

def find_nearest_neighbors(X, k=1):
    # Students.
    pass

In [26]:
vanilla_ae_emb_file = 'data/out/Neural_nets/vanilla_ae/latent_codes.npz'
part_ae_emb_file = 'data/out/Neural_nets/pc_aware_ae/latent_codes.npz'
golden_part_dist_file = 'data/in/golden_dists.npz'

golden_data = np.load(golden_part_dist_file)
golden_part_dist = golden_data['golden_part_dist']
golden_names = golden_data['golden_names']

# To load vanilla-AE-embeddings (if False will open those of the 2-branch AE).
vanilla = True

In [27]:
#  Students. Extract shape-net model ids of golden, map them to their parts.
sn_id_to_parts = defaultdict(list)
for name in golden_names:
    test = name.split("_")
    sn_id, part_id = test[0], test[2]
    sn_id_to_parts[sn_id].append(part_id)

In [28]:
# Students. Map shape-net model id and part_id to location in distance matrix.
id_to_part_loc = dict()
for sn_id, part_id in sn_id_to_parts.items():
    print(sn_id, part_id) # golden_part_dist
    
    # For each shape you match it with a neighbor and 
    # use the "golden" distances 
    # to define a similarity/distance score between them. Please report the average of those scores, wrt. the number of test shapes. (total_sum / n_shapes)



9ab18a33335373b2659dda512294c744 ['2', '3', '1']
ff2223a085d32243696b74614952b2d0 ['3', '1', '2', '4']
3c58dc00cf251172c05575120a46cd3b ['1', '2', '3']
88aea5f475654e491d77ac5988aa9265 ['2', '1', '3']
d36ecfec3f3c18a9a28fdea8831e592e ['1', '2', '3']
74cc57ea0e2e06dbe4106b1d06dc89b3 ['1', '2', '3']
6e21df8cb8db3819899718fd4553951 ['1', '2', '3']
f2ef238e2e4615e92cf8e80367b40c9a ['2', '3', '1']
a9eae610d9ab7c40a7d147d66b8f507d ['3', '1', '2']
40ab45735f526717e25492f5f62d416f ['3', '1', '4', '2']
4ce5a0d60fc942f2e595afbdc333be49 ['1', '3', '2', '4']
3799a4d787d31c0bc580fdeb5460f6d6 ['1', '3', '2']
640f61579181aef13ad3591a780fa12b ['1', '3', '2']
9570ea7fa3a0ef775436c76691cf3d3 ['2', '4', '3', '1']
9915f9247f671893b89b3e01b90fe955 ['2', '3', '1']
e08add655395ce8df51f77a6d7299806 ['3', '4', '2', '1']
632a722f9edbaf607b11ae648ea92233 ['1', '3', '2']
c951e7bd4c0cbce17ec5a98b3b8c425f ['1', '2', '3']
cd5ad4afabaed0d3e762624dc3c8fa2a ['3', '1', '2']
79c9fdc6bba82709c8687ff9b0b4e4ac ['1', '3', '2

In [ ]:
if vanilla:
    in_d = np.load(vanilla_ae_emb_file)    # Students: this assumes that you used np.savez 
    latent_codes = in_d['l_codes']         # in the end of establishing_aes.py to save the
    test_names = in_d['test_names']        # latent_codes/test_names.
else:
    in_d = np.load(part_ae_emb_file)
    latent_codes = in_d['l_codes']
    test_names = in_d['test_names']


# Students. Implement Question [g].


print('Average Holistic distance:', 0)
print('Average Aggrement: ', 0)
print('Average Cumulative Dists:', 0)

# Tensorflow pipeline code

In [2]:
# Students: If you run your code with Python3 instead of Python2 (like in Azzure) set python_2to3=True
python_2to3 = True


# Students: Default options for splits. Do NOT change.
split_loads = [0.75, 0.15, 0.10]
random_seed = 42
verbose = True
n_total_shapes = 1000

# Training options.
do_training = True
batch_size = 50        # Students: do NOT change this one.
held_out_step = 10
n_epochs = 400

# Loading DATA/employing train/val/test splits.
generator = unpickle_data('data/in/part_labeled_point_clouds.pkl', python_2to3)
hw4_data = six.next(generator)
sids = six.next(unpickle_data('data/in/randomized_ids.pkl', python_2to3))
hw4_data.part_masks = hw4_data.part_masks.astype(np.int32)

hw4_data = hw4_data.subsample(n_total_shapes, replace=False, seed=random_seed)
net_data = {}
for s in sids:
    idx = sids[s]
    net_data[s] = hw4_data.extract(idx)
test_data = net_data['test'].freeze()

In [3]:
tf.reset_default_graph()

In [4]:
# Define Configuration of the Point-AE.
pc_ae_conf = Neural_Net_Conf()

n_pc_per_model = net_data['train'].pcs.shape[-2]
bneck = 128

pc_ae_conf.encoder = encoder_with_convs_and_symmetry
pc_ae_conf.decoder = decoder_with_fc_only

pc_ae_conf.n_points = n_pc_per_model

pc_ae_conf.encoder_args = {'n_filters':5, 'filter_sizes':[32, 64, 64, 128, 128],
                          'verbose':True} # Students add your encoder's options.

pc_ae_conf.decoder_args = {'layer_sizes': [256, 256, n_pc_per_model * 3],                           
                           'verbose': True}

pc_ae_conf.learning_rate = 0.0009
pc_ae_conf.saver_max_to_keep = 1
pc_ae_conf.allow_gpu_growth = True


# Students: Will it predict part-segmentation too? If so, set to true. 
pc_ae_conf.use_parts = True
pc_ae_conf.n_parts = 4
pc_ae_conf.part_pred_with_one_layer = True


# How much is the relative importance of part-prediction vs. pc-reconstruction.
pc_ae_conf.part_weight = 0.005 #Students: leave this option unchanged for the (non-bonus) questions.

if pc_ae_conf.use_parts:
    if pc_ae_conf.part_pred_with_one_layer:
        pc_ae_conf.name = 'pc_aware_ae'
    else:
        pc_ae_conf.name = 'pc_aware_ae_bonus'
    n_losses = 2
else:
    pc_ae_conf.name = 'vanilla_ae'
    n_losses = 1

In [ ]:
ae = cs233PointAutoEncoder(pc_ae_conf.name, pc_ae_conf)

Building Encoder
Building Decoder


In [ ]:
if do_training:
    save_dir = create_dir(osp.join('data/out/Neural_nets', pc_ae_conf.name))    
    tf.set_random_seed(random_seed)
    np.random.seed(random_seed)
    with open(osp.join(save_dir, 'net_stats.txt'), 'w') as file_out:            
        train_loss, val_loss, test_loss = ae.train_model(net_data, n_epochs, batch_size, save_dir,\
                                                         held_out_step, fout=file_out)

Training epoch/loss/duration:  1 [  32.79234867 1384.23806966] 4.6812920570373535
Training epoch/loss/duration:  2 [   5.9429807  1340.50447591] 4.2917585372924805
Training epoch/loss/duration:  3 [   4.31091208 1313.40863444] 4.314695358276367
Training epoch/loss/duration:  4 [   3.96747855 1289.99545898] 4.3052709102630615
Training epoch/loss/duration:  5 [   3.82053219 1266.81861979] 4.319814443588257
Training epoch/loss/duration:  6 [   3.74555233 1244.69735514] 4.339019536972046
Training epoch/loss/duration:  7 [   3.66161928 1219.06887207] 4.307007789611816
Training epoch/loss/duration:  8 [   3.60792726 1187.16700846] 4.293313026428223
Training epoch/loss/duration:  9 [   3.52158537 1145.24928385] 4.282711505889893
Training epoch/loss/duration:  10 [   3.46649184 1088.4784668 ] 4.323501825332642
Val/Test epoch/loss: 10 [   3.56902385 1063.39727783] [   3.80908132 1063.96118164]
Training epoch/loss/duration:  11 [   3.36831381 1017.92478841] 4.292316675186157
Training epoch/loss/

Training epoch/loss/duration:  95 [  1.1459022  292.13413798] 4.343839645385742
Training epoch/loss/duration:  96 [  1.16546574 291.5501475 ] 4.328896522521973
Training epoch/loss/duration:  97 [  1.17919745 289.83743693] 4.318761825561523
Training epoch/loss/duration:  98 [  1.16409356 288.51599528] 4.317764043807983
Training epoch/loss/duration:  99 [  1.15703775 287.07625631] 4.329935073852539
Training epoch/loss/duration:  100 [  1.14899754 285.95674845] 4.326789379119873
Val/Test epoch/loss: 100 [  1.50413638 289.62808228] [  1.54378788 325.03894043]
Training epoch/loss/duration:  101 [  1.11484833 284.29006653] 4.347994327545166
Training epoch/loss/duration:  102 [  1.12753127 283.69942525] 4.320740699768066
Training epoch/loss/duration:  103 [  1.13216595 282.24488627] 4.310727596282959
Training epoch/loss/duration:  104 [  1.13258379 281.42247009] 4.321856260299683
Training epoch/loss/duration:  105 [  1.10001508 280.55611877] 4.325891017913818
Training epoch/loss/duration:  10

# When use_parts = False

In [ ]:
# Load model on optimal (per validation) epoch.
epoch_to_restore = (1+np.argmin(val_loss))*held_out_step # Student 
ae.restore_model(save_dir, epoch_to_restore, verbose=True)



In [ ]:
plt.figure(figsize=(12,8))

epochs_plotted = held_out_step*np.arange(1,int(n_epochs/held_out_step)+1)
train_loss_10_epoch = train_loss[epochs_plotted-1]

plt.plot(epochs_plotted,train_loss_10_epoch/15.0, label='Train loss per sample')
plt.plot(epochs_plotted,val_loss/3., label='Val loss per sample')
plt.plot(epochs_plotted,test_loss/2., label='Test loss per sample')


plt.title('Average reconstruction loss in the training, dev and test sets')
plt.legend()
plt.show()

In [ ]:
feed = ae.prepare_feed(test_data, test_data.n_examples)
latent_test_vectors = ae.sess.run(ae.z, feed_dict=feed)

In [ ]:
test_names = test_data.model_names
feed = ae.prepare_feed(test_data, test_data.n_examples)

l_codes = ae.sess.run(ae.z, feed_dict=feed) # Students: compute this.
np.savez(osp.join(save_dir, 'latent_codes'), l_codes=l_codes, test_names=test_names)

In [ ]:
# Students: Save-plot reconstructions.
n_plots = 5
in_pc = test_data.pcs[:n_plots]
in_names = test_data.model_names[:n_plots]
in_masks = test_data.part_masks[:n_plots]

recon_pc = ae.sess.run(ae.pc_reconstr, feed_dict=feed)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
in_pc[0].shape

In [ ]:
for n_th_shape in range(5):
    fig = plt.figure(figsize=(15,8))

    ax = Axes3D(fig)

    ax.scatter(in_pc[n_th_shape,:,0], in_pc[n_th_shape,:,1], in_pc[n_th_shape,:,2], c='red')
    ax.scatter(recon_pc[n_th_shape,:,0], recon_pc[n_th_shape,:,1], recon_pc[n_th_shape,:,2], c='blue')

    plt.show()

In [ ]:
random_seed = 42 # Students use this seed if you use sklearn's TSNE with default parameters.
vanilla_ae_emb_file = 'data/out/Neural_nets/vanilla_ae/latent_codes.npz'

In [ ]:
in_d = np.load(vanilla_ae_emb_file)    # Students: this assumes that you used np.savez 
latent_codes = in_d['l_codes']         # in the end of establishing_aes.py
test_names = in_d['test_names']

# Students: Compute 2D TSNE
tsne_lcodes = TSNE(n_components=2, random_state=random_seed).fit_transform(latent_codes)

In [ ]:
im_files = []
top_im_dir = 'data/in/images/'
for name in test_names:
    im_files.append(osp.join(top_im_dir, str(name)[2:-1] + '.png'))

In [ ]:
plot_2d_embedding_in_grid_greedy_way(tsne_lcodes, im_files, big_dim=1500, small_dim=70, 
                                     save_file='data/out/Neural_nets/vanilla_ae/test_pc_tsne.png', transparent=True)

In [ ]:
plt.figure(figsize=(20,15))
img = plt.imread('data/out/Neural_nets/vanilla_ae/test_pc_tsne.png')
plt.imshow(img)
plt.show()